Indexing Metrics

Indexing everything will slow down the Inserts and make Inserts expensive

Composite Index can help you with Order BY and WHERE with many filters

Global Secondary Indexes

Indexing Metrics

select * from c 
where c.OwnerUserId = 200 
and c.PostType ='Question' 
and c.Score > 10

GSI = Cross Partition's nightmare

Data gets sync automatically

GSI has its own structure (pkey, data model, indexing)




In [1]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;

Installed Packages Microsoft.Azure.Cosmos, 3.53.1 Newtonsoft.Json, 13.0.4

In [4]:
var endpoint = "https://localhost:8081";
var key ="C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(endpoint, key);
var db = client.GetDatabase("StackOverflow");
var postContainer = db.GetContainer("Posts");

In [5]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
}

In [11]:
var id = 1;
var productQuery = new QueryDefinition("select * from c where c.OwnerUserId = 200 and c.PostType ='Question' and c.Score > 10");
    //.WithParameter("@id", id);
    var options = new QueryRequestOptions() { PopulateIndexMetrics = true };
var iterator = postContainer.GetItemQueryIterator<Post>(productQuery, requestOptions: options);
var results = new List<Post>();

while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
    Console.WriteLine($"Request charge: {response.RequestCharge}");
    response.IndexMetrics.Display();
}
results.Display();

Request charge: 3.17


{"UtilizedIndexes":{"SingleIndexes":[{"IndexSpec":"/PostType/?"},{"IndexSpec":"/Score/?"},{"IndexSpec":"/OwnerUserId/?"}],"CompositeIndexes":[]},"PotentialIndexes":{"SingleIndexes":[],"CompositeIndexes":[{"IndexSpecs":["/OwnerUserId ASC","/PostType ASC","/Score ASC"],"IndexImpactScore":"High"}]}}

index value 0 Submission#5+Post id d5121c9c-e4d4-4a55-ae32-afe2820b998b PostId 898 PostBody <p>How have you implemented Internationalization (i18n) in actual projects you've worked on?</p>

<p>I took an interest in making software cross-cultural after I read the famous post by Joel, <a href="http://www.joelonsoftware.com/articles/Unicode.html" rel="noreferrer">The Absolute Minimum Every Software Developer Absolutely, Positively Must Know About Unicode and Character Sets (No Excuses!)</a>. However, I have yet to able to take advantage of this in a real project, besides making sure I used Unicode strings where possible. But making all your strings Unicode and ensuring you understand what encoding everything you work with is in is just the tip of the i18n iceberg.</p>

<p>Everything I have worked on to date has been for use by a controlled set of US English speaking people, or i18n just wasn't something we had time to work on before pushing the project live. So I am looking for any tips or war stories people have about making software more localized in real world projects.</p>
 Title Internationalization in your projects ViewCount 4279 AnswerCount 11 CommentCount 1 FavoriteCount 36 AcceptedAnswerId 0 CreatedOn 2008-08-04 00:08:51Z ClosedDate <null> OwnerUserId 200 OwnerDisplayName John Downey PostType Question Score 44 Tags <language-agnostic><unicode><localization><internationalization>